This code is derived from code at:

https://public.paws.wmcloud.org/User:Jtmorgan/data512_a1_example.ipynb


In [92]:
import json
import requests
import pandas

The following strings represent the HTTP endpoints for the two REST APIs that we'll use.

The parameters for the APIs are encoded in these strings and are represented below by the URL segments that are in braces, such as `{granularity}`. These are placeholders that are replaced by values that we specify in blocks of JSON and pass into the function that makes the API call. (That function is defined later in this file.)

In [93]:
endpoint_pageviews = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/aggregate/{project}/{access}/{agent}/{granularity}/{start}/{end}'

endpoint_legacy = 'https://wikimedia.org/api/rest_v1/metrics/legacy/pagecounts/aggregate/{project}/{access-site}/{granularity}/{start}/{end}'

In this section, we specify blocks of JSON that contain the parameters for each of the **five** API calls that we make.

The first **three** parameter blocks are for the Pageviews API. The following **two** parameter blocks are for the Pagecount (legacy) API.

In [94]:
#
# Parameters for getting aggregated current standard pageview data
#
# See: https://wikimedia.org/api/rest_v1/#!/Pageviews_data/get_metrics_pageviews_aggregate_project_access_agent_granularity_start_end
#
params_pageviews_desktop = {
                    "project" : "en.wikipedia.org",
                    "access" : "desktop",
                    "agent" : "user",
                    "granularity" : "monthly",
                    "start" : "2015070100",
                    # for end use 1st day of month following final month of data
                    "end" : '2020100100'
                        }

params_pageviews_mobile_app = {
                    "project" : "en.wikipedia.org",
                    "access" : "mobile-app",
                    "agent" : "user",
                    "granularity" : "monthly",
                    "start" : "2015070100",
                    # for end use 1st day of month following final month of data
                    "end" : '2020100100'
                        }

params_pageviews_mobile_web = {
                    "project" : "en.wikipedia.org",
                    "access" : "mobile-web",
                    "agent" : "user",
                    "granularity" : "monthly",
                    "start" : "2015070100",
                    # for end use 1st day of month following final month of data
                    "end" : '2020100100'
                        }

#
# Parameters for getting aggregated legacy view data 
#
# See: https://wikimedia.org/api/rest_v1/#!/Legacy_data/get_metrics_legacy_pagecounts_aggregate_project_access_site_granularity_start_end
#
params_pagecount_legacy_desktop = {
                 "project" : "en.wikipedia.org",
                 "access-site" : "desktop-site",
                 "granularity" : "monthly",
                 "start" : "2008010100",
                 # for end use 1st day of month following final month of data
                 "end" : "2016080100"
                    }

params_pagecount_legacy_mobile = {
                 "project" : "en.wikipedia.org",
                 "access-site" : "mobile-site",
                 "granularity" : "monthly",
                 "start" : "2008010100",
                 # for end use 1st day of month following final month of data
                 "end" : "2016080100"
                    }


In [ ]:
#
# Identify myself in the headers of the HTTP request
#
headers = {
    'User-Agent': 'https://github.com/carljparker',
    'From': 'cajopa@uw.edu'
}

In [95]:
def api_call(endpoint,parameters):
    call = requests.get(endpoint.format(**parameters), headers=headers)
    response = call.json()
    
    return response

In [96]:
monthly_pageviews = api_call(endpoint_pageviews, params_pageviews)

In [97]:
type( monthly_pageviews )

dict

In [98]:
type( monthly_pageviews['items'] )

list

In [99]:
with open("pageviews_desktop_201507-202009.json", "w") as write_file:
    json.dump(monthly_pageviews, write_file)

In [100]:
pandas.read_json( json.dumps( monthly_pageviews[ 'items' ] ), orient='records', convert_dates = False )

,project,access,agent,granularity,timestamp,views
0,en.wikipedia,desktop,user,monthly,2015070100,4376666686
1,en.wikipedia,desktop,user,monthly,2015080100,4332482183
2,en.wikipedia,desktop,user,monthly,2015090100,4485491704
3,en.wikipedia,desktop,user,monthly,2015100100,4477532755
4,en.wikipedia,desktop,user,monthly,2015110100,4287720220
...,...,...,...,...,...,...
58,en.wikipedia,desktop,user,monthly,2020050100,3078093615
59,en.wikipedia,desktop,user,monthly,2020060100,2721328557
60,en.wikipedia,desktop,user,monthly,2020070100,2638936132
61,en.wikipedia,desktop,user,monthly,2020080100,2613058239


In [101]:
monthly_pagecount_legacy = api_call(endpoint_legacy, params_pagecount_legacy)

In [102]:
pandas.read_json( json.dumps( monthly_pagecount_legacy[ 'items' ] ), orient='records', convert_dates = False )

,project,access-site,granularity,timestamp,count
0,en.wikipedia,desktop-site,monthly,2008010100,4930902570
1,en.wikipedia,desktop-site,monthly,2008020100,4818393763
2,en.wikipedia,desktop-site,monthly,2008030100,4955405809
3,en.wikipedia,desktop-site,monthly,2008040100,5159162183
4,en.wikipedia,desktop-site,monthly,2008050100,5584691092
...,...,...,...,...,...
98,en.wikipedia,desktop-site,monthly,2016030100,5407676056
99,en.wikipedia,desktop-site,monthly,2016040100,5572235399
100,en.wikipedia,desktop-site,monthly,2016050100,5330532334
101,en.wikipedia,desktop-site,monthly,2016060100,4975092447
